In [1]:
import os
print(os.getcwd())

c:\Users\Frank\Sherlock-Hemlock\data


In [1]:
# from transformers import AutoTokenizer, AutoModel
# import torch

# # Load pre-trained model tokenizer (vocabulary)
# tokenizer = AutoTokenizer.from_pretrained("camembert-base")

# # Load pre-trained model (weights)
# model = AutoModel.from_pretrained("camembert-base")

# # Text to encode
# text = "Je suis un étudiant."

# # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
# input_ids = tokenizer.encode(text, add_special_tokens=True) 
# tokens = tokenizer.convert_ids_to_tokens(input_ids) # Convert the ids to tokens.

# # Create a tensor out of the input ids and move it to the GPU if available
# input_ids = torch.tensor([input_ids])

# # If you have a GPU, put everything on cuda
# #input_ids = input_ids.to('cuda')
# #model.to('cuda')

# # This context manager makes sure that the model's internal states such as the 
# # computation graph, the gradients, etc... are not saved in memory which would 
# # lead to increased memory usage.
# with torch.no_grad():
#     last_hidden_states = model(input_ids)[0]  # Models outputs are now tuples

# # Get the embeddings of the [CLS] token for each layer
# word_embeddings = last_hidden_states[0]

c:\Users\Frank\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Frank\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Frank\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see ac

In [5]:
# from transformers import CamembertTokenizer, CamembertModel
# import torch

# tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
# model = CamembertModel.from_pretrained('camembert-base')

# def get_word_embedding(word):
#     input_ids = tokenizer.encode(word, return_tensors='pt')
#     with torch.no_grad():
#         outputs = model(input_ids)
#     # Use the mean of the last hidden state as the word embedding
#     word_embedding = outputs.last_hidden_state.mean(dim=1)
#     return word_embedding

# word = "maison"
# embedding = get_word_embedding(word)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [41]:
import gensim
import gensim.models
import ast
from spacy.lang.fr.stop_words import STOP_WORDS

model = gensim.models.fasttext._load_fasttext_format("../cc.fr.2023.bin", full_model=False)

In [71]:
import pandas as pd
articles = pd.read_csv("../../local_datasets/bsard_extra/bsard_articles_preprocessed.csv")
q_test = pd.read_csv("../../local_datasets/bsard_extra/bsard_expert_questions_test_preprocessed.csv").drop('Unnamed: 0', axis=1)
q_train = pd.read_csv("../../local_datasets/bsard_extra/bsard_expert_questions_train_preprocessed.csv").drop('Unnamed: 0', axis=1)
q_synth = pd.read_csv("../../local_datasets/bsard_extra/bsard_synthetic_questions_preprocessed.csv").drop('Unnamed: 0', axis=1)

articles["tokens"] = articles["tokens"].apply(ast.literal_eval)
q_train["tokens"] = q_train["tokens"].apply(ast.literal_eval)
q_test["tokens"] = q_test["tokens"].apply(ast.literal_eval)
q_synth["tokens"] = q_synth["tokens"].apply(ast.literal_eval)

In [50]:
from spacy.lang.fr.stop_words import STOP_WORDS
exclude_list = ["l'", "le", "la", "les", ".", ",", ";", ":", "(", ")", "[", "]", "{", "}", "?", "!", "-", "«", "»"]

def generate_sentence_embeddings(tokenized_sentences, exclude_list, fasttext_model):
    sentence_embeddings = []
    for sentence in tokenized_sentences:
        word_embeddings = []
        for idx, word in enumerate(sentence):
            word = word.lower()
            if word not in exclude_list and word not in STOP_WORDS and word.isalpha():
                word_embedding = fasttext_model.wv[word]
                word_embeddings.append((idx, word, word_embedding))
        sentence_embeddings.append(word_embeddings)
    return sentence_embeddings

In [75]:
q_train['word_embeddings'] = q_train['tokens'].apply(lambda tokens: generate_sentence_embeddings(tokens, exclude_list, model))
q_test['word_embeddings'] = q_test['tokens'].apply(lambda tokens: generate_sentence_embeddings(tokens, exclude_list, model))
q_synth['word_embeddings'] = q_synth['tokens'].apply(lambda tokens: generate_sentence_embeddings(tokens, exclude_list, model))

In [77]:
articles['word_embeddings'] = articles['tokens'].apply(lambda tokens: generate_sentence_embeddings(tokens, exclude_list, model))

In [87]:
articles['word_embeddings']

0        [[(1, présent, [ 5.80670917e-03  9.87947453e-0...
1        [[(1, présent, [ 5.80670917e-03  9.87947453e-0...
2        [[(1, présent, [ 5.80670917e-03  9.87947453e-0...
3        [[(1, sens, [ 0.08547884  0.08123539  0.016227...
4        [[(1, plan, [-0.07429864  0.03693126  0.016701...
                               ...                        
22628    [[(1, ville, [-4.35092673e-02  5.73099628e-02 ...
22629    [[(1, pouvoir, [ 6.06184416e-02  4.02655900e-0...
22630    [[(0, aucune, [ 0.02575823  0.10380266  0.0596...
22631    [[(2, régence, [-0.04805386  0.07182669 -0.106...
22632    [[(2, commun, [ 9.44962166e-03 -2.95859687e-02...
Name: word_embeddings, Length: 22633, dtype: object

In [95]:
articles.to_hdf("../../local_datasets/bsard_extra/gsard_articles.h5", key='dataframe', mode='w', complevel=9, complib='blosc')
q_train.to_hdf("../../local_datasets/bsard_extra/gsard_expert_questions_train.h5", key='dataframe', mode='w', complevel=9, complib='blosc')
q_test.to_hdf("../../local_datasets/bsard_extra/gsard_expert_questions_test.h5", key='dataframe', mode='w', complevel=9, complib='blosc')
q_synth.to_hdf("../../local_datasets/bsard_extra/gsard_expert_questions_synth.h5", key='dataframe', mode='w', complevel=9, complib='blosc')

C:\Users\Frank\AppData\Local\Temp\ipykernel_11636\2887933592.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['article', 'pos', 'dep', 'heads', 'tokens', 'word_embeddings'], dtype='object')]

  articles.to_hdf("../../local_datasets/bsard_extra/gsard_articles.h5", key='dataframe', mode='w', complevel=9, complib='blosc')
C:\Users\Frank\AppData\Local\Temp\ipykernel_11636\2887933592.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['category', 'subcategory', 'question', 'extra_description',
       'article_ids', 'normalized_question', 'pos', 'dep', 'heads', 'tokens',
       'word_embeddings'],
      dtype='object')]

  q_train.to_hdf("../../local_datasets/bsard_extra/gsard_expert_questions_train.h5", key='dataframe', 